This code is a duplicate of the Mistral parsing but modified for the GPT API, to use when Mistral fails. In this case it's for the last 30/34000 entries. 

In [20]:
import os
import json
import psycopg2
import requests
import time
import threading
from tqdm import tqdm
from dotenv import load_dotenv
from queue import Queue, Empty
from concurrent.futures import ThreadPoolExecutor

# Load environment variables
load_dotenv()

# Ensure required environment variables are loaded
REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "POSTGRES_PASSWORD", "POSTGRES_DB"]
for var in REQUIRED_ENV_VARS:
    if not os.getenv(var):
        raise EnvironmentError(f"❌ Missing required environment variable: {var}")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
BATCH_SIZE = 5  # Keep small to reduce cost
MAX_WORKERS = 5

# Toggle to start from last uploaded entry
START_FROM_LAST_PROCESSED = False
LAST_PROCESSED_ID = 3859

print("🔑 API Key Loaded:")
print(f"OPENAI_API_KEY: {OPENAI_API_KEY[-6:]}")

# PostgreSQL Connection
try:
    conn = psycopg2.connect(
        dbname=os.getenv("POSTGRES_DB"),
        user="rohansharma",
        password=os.getenv("POSTGRES_PASSWORD"),
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    print("✅ Successfully connected to PostgreSQL!")
except psycopg2.Error as e:
    print(f"❌ Failed to connect to PostgreSQL: {e}")
    exit(1)

# Count total unprocessed entries
try:
    if START_FROM_LAST_PROCESSED:
        cursor.execute(f"SELECT COUNT(*) FROM sep_embeddings WHERE id >= {LAST_PROCESSED_ID} AND mistral_output IS NULL;")
    else:
        cursor.execute("SELECT COUNT(*) FROM sep_embeddings WHERE mistral_output IS NULL;")

    total_entries = cursor.fetchone()[0]
    print(f"📊 Total unprocessed entries: {total_entries}")
except psycopg2.Error as e:
    print(f"❌ Error executing query: {e}")
    exit(1)

# Fetch unprocessed rows
if START_FROM_LAST_PROCESSED:
    cursor.execute(f"""
        SELECT id, content FROM sep_embeddings 
        WHERE id >= {LAST_PROCESSED_ID} AND mistral_output IS NULL 
        ORDER BY id;
    """)
else:
    cursor.execute("""
        SELECT id, content FROM sep_embeddings 
        WHERE mistral_output IS NULL 
        ORDER BY id;
    """)

rows = cursor.fetchall()
batch_updates = []

# Initialize progress bar
progress_bar = tqdm(total=total_entries, desc="Processing Entries", unit="entry")

# Initialize queue
task_queue = Queue()
response_queue = Queue()

# Enqueue tasks
for entry_id, content in rows:
    task_queue.put((entry_id, content))


🔑 API Key Loaded:
OPENAI_API_KEY: Q_HFEA
✅ Successfully connected to PostgreSQL!
📊 Total unprocessed entries: 4


Processing Entries:   0%|          | 0/7 [01:18<?, ?entry/s]


In [ ]:
import json
import re

def extract_first_valid_json(api_content):
    """Extracts only the first valid JSON object from the response content with improved escape handling."""
    print(f"\n🔍 Raw API Content received:\n{api_content[:200]}...")
    
    # First attempt: Try to parse with proper escape handling
    try:
        # Replace problematic escape sequences
        cleaned_content = api_content.replace('\\n', '\n').replace('\\r', '\r').replace('\\t', '\t')
        # Handle any remaining invalid escapes by replacing them with raw backslashes
        cleaned_content = cleaned_content.replace('\\', '\\\\')
        # Now remove any double backslashes we might have created
        cleaned_content = cleaned_content.replace('\\\\\\\\', '\\\\')
        
        parsed = json.loads(cleaned_content)
        print("✅ Successfully parsed complete response as JSON")
        return parsed
    except json.JSONDecodeError as e:
        print(f"ℹ️ First parsing attempt failed: {str(e)}")
    
    # Second attempt: Try to extract and parse just the JSON structure
    try:
        start = api_content.find('{')
        end = api_content.rfind('}') + 1
        
        if start >= 0 and end > start:
            potential_json = api_content[start:end]
            # Apply the same cleaning to the extracted JSON
            potential_json = potential_json.replace('\\n', '\n').replace('\\r', '\r').replace('\\t', '\t')
            potential_json = potential_json.replace('\\', '\\\\')
            potential_json = potential_json.replace('\\\\\\\\', '\\\\')
            
            print(f"\n🔍 Attempting to parse extracted JSON:\n{potential_json[:200]}...")
            
            parsed = json.loads(potential_json)
            print("✅ Successfully parsed extracted JSON")
            return parsed
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse extracted JSON: {str(e)}")
    
    print("❌ No valid JSON found in response")
    return None

def call_gpt_api(api_key, entry_id, content, attempt=0):
    """Sends an API request to GPT with improved JSON handling."""
    print(f"\n🎯 Starting API call for entry {entry_id}")
    
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Modified system message to prevent escape character issues
    messages = [
        {"role": "system", "content": "Extract structured philosophical knowledge. Return a single valid JSON object with no special characters or escape sequences in text fields."},
        {"role": "user", "content": f"""
        Extract philosophical knowledge in structured format.
        Return a single JSON object.
        Avoid special characters or escape sequences in text.
        
        Input text:
        {content}
        
        Output Format:
        {{
            "category": "thinker" | "concept" | "era",
            "metadata": {{
                "name": "...",
                "description": "...",
                "time_period": "..."
            }},
            "key_beliefs": [
                {{"belief": "...", "justification": "...", "related_concepts": ["...", "..."]}}
            ],
            "key_concepts": [
                {{"name": "...", "definition": "...", "related_fields": ["...", "..."]}}
            ],
            "associated_thinkers": ["...", "..."],
            "associated_eras": ["...", "..."]
        }}
        """}
    ]
    
    data = {
        "model": "gpt-4-turbo",
        "messages": messages,
        "temperature": 0.4
    }
    
    print(f"⏳ Making API request for entry {entry_id}...")
    time.sleep(1.5)
    
    response = requests.post(url, headers=headers, json=data)
    print(f"📡 Received response for entry {entry_id} (Status: {response.status_code})")
    
    if response.status_code == 200:
        try:
            parsed_response = response.json()
            api_content = parsed_response["choices"][0]["message"]["content"]
            
            print(f"🔍 Attempting to extract JSON for entry {entry_id}")
            structured_output = extract_first_valid_json(api_content)
            
            if structured_output:
                print(f"✅ Successfully extracted JSON for entry {entry_id}")
                return entry_id, structured_output
            else:
                print(f"❌ No valid JSON found for entry {entry_id}")
                return None
                
        except json.JSONDecodeError as e:
            print(f"❌ Failed to parse API response for entry {entry_id}: {str(e)}")
            print(f"Raw response:\n{response.text[:500]}...")
            return None
            
    elif response.status_code == 429:
        if attempt < 4:
            wait_time = 5 * (2 ** attempt)
            print(f"⏳ Rate limited (429) for entry {entry_id}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            return call_gpt_api(api_key, entry_id, content, attempt + 1)
        else:
            print(f"❌ Entry {entry_id} failed after 4 retries due to rate limiting")
            return None
            
    else:
        print(f"❌ API request failed for entry {entry_id}: {response.status_code}")
        print(f"Error response:\n{response.text[:500]}...")
        return None
    """Sends an API request to GPT with exponential backoff for 429 errors."""
    print(f"\n🎯 Starting API call for entry {entry_id}")
    
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    messages = [
        {"role": "system", "content": "Extract structured philosophical knowledge. Your response should always be a single valid JSON object."},
        {"role": "user", "content": f"""
        You are an AI tasked with extracting structured philosophical knowledge from a given text.
        Your goal is to **categorize the entry and extract key information in a structured format.**
        Be **as detailed as possible while remaining concise.**
        **Always return a single JSON object, never multiple JSON objects.**
        
        ## **Input:**
        {content}
        
        ## **Output Format (JSON):**
        {{
            "category": "thinker" | "concept" | "era",
            "metadata": {{
                "name": "...",
                "description": "...",
                "time_period": "..."
            }},
            "key_beliefs": [
                {{"belief": "...", "justification": "...", "related_concepts": ["...", "..."]}}
            ],
            "key_concepts": [
                {{"name": "...", "definition": "...", "related_fields": ["...", "..."]}}
            ],
            "associated_thinkers": ["...", "..."],
            "associated_eras": ["...", "..."]
        }}
        
        **NEVER return multiple JSON objects.**
        **NO output except for the JSON.**
        """}
    ]
    
    data = {
        "model": "gpt-4-turbo",
        "messages": messages,
        "temperature": 0.4
    }
    
    print(f"⏳ Making API request for entry {entry_id}...")
    time.sleep(2)  # Light rate-limiting for cost efficiency
    
    response = requests.post(url, headers=headers, json=data)
    print(f"📡 Received response for entry {entry_id} (Status: {response.status_code})")
    
    if response.status_code == 200:
        try:
            parsed_response = response.json()
            api_content = parsed_response["choices"][0]["message"]["content"]
            
            print(f"🔍 Attempting to extract JSON for entry {entry_id}")
            structured_output = extract_first_valid_json(api_content)
            
            if structured_output:
                print(f"✅ Successfully extracted JSON for entry {entry_id}")
                return entry_id, structured_output
            else:
                print(f"❌ No valid JSON found for entry {entry_id}")
                return None
                
        except json.JSONDecodeError:
            print(f"❌ Failed to parse API response for entry {entry_id}")
            print(f"Raw response:\n{response.text[:500]}...")  # Print first 500 chars
            return None
            
    elif response.status_code == 429:
        if attempt < 4:
            wait_time = 5 * (2 ** attempt)  # Exponential backoff
            print(f"⏳ Rate limited (429) for entry {entry_id}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            return call_gpt_api(api_key, entry_id, content, attempt + 1)
        else:
            print(f"❌ Entry {entry_id} failed after 4 retries due to rate limiting")
            return None
            
    else:
        print(f"❌ API request failed for entry {entry_id}: {response.status_code}")
        print(f"Error response:\n{response.text[:500]}...")  # Print first 500 chars
        return None

In [22]:
def api_worker():
    """Worker that fetches requests from the queue and processes them."""
    while not task_queue.empty():
        try:
            entry_id, content = task_queue.get(timeout=10)
            print(f"🔄 Processing entry {entry_id}")

            # Add detailed logging for API call
            print(f"📡 Calling API for entry {entry_id}")
            response = call_gpt_api(OPENAI_API_KEY, entry_id, content)
            
            if response:
                entry_id, structured_output = response
                print(f"✨ Got API response for {entry_id}: {str(structured_output)[:100]}...")
                response_queue.put((entry_id, structured_output))
                print(f"✅ Successfully added entry {entry_id} to response queue!")
            else:
                print(f"❌ API call failed for entry {entry_id} - returned None")

            task_queue.task_done()

        except Empty:
            print("📭 API worker queue empty. Exiting thread.")
            break
        except Exception as e:
            print(f"❌ Unexpected error in api_worker for entry {entry_id}: {str(e)}")
            task_queue.task_done()

def process_responses():
    """Processes the completed API responses and updates the database."""
    if conn.closed:
        print("❌ Database connection is closed!")
        return

    try:
        conn.isolation_level  # Ensure the connection is still active
    except psycopg2.OperationalError:
        print("❌ Lost database connection!")
        return

    batch_updates = []
    batch_count = 0
    processed_count = 0

    print("🚀 Response processor started.")

    while True:
        try:
            print(f"⏳ Waiting for responses... (Current batch size: {len(batch_updates)})")
            print(f"📊 Stats - Processed: {processed_count}, Batches: {batch_count}")
            
            entry = response_queue.get(timeout=30)  # Increased timeout
            
            if entry is None:
                print("⚠️ Received None entry, skipping...")
                continue

            entry_id, mistral_output_json = entry
            processed_count += 1
            print(f"💾 Processing database update for entry {entry_id}")

            if entry_id and mistral_output_json:
                batch_updates.append((json.dumps(mistral_output_json), entry_id))
                print(f"📦 Added entry {entry_id} to batch (Batch size: {len(batch_updates)})")

            if len(batch_updates) >= BATCH_SIZE:
                try:
                    print(f"🚀 Uploading batch #{batch_count + 1} ({len(batch_updates)} entries)...")

                    cursor.executemany("""
                        UPDATE sep_embeddings 
                        SET mistral_output = %s 
                        WHERE id = %s;
                    """, batch_updates)
                    conn.commit()

                    rows_updated = cursor.rowcount
                    batch_count += 1
                    print(f"✅ Batch #{batch_count} uploaded ({rows_updated} rows)")
                    batch_updates = []

                except Exception as e:
                    print(f"❌ Database error during batch upload: {str(e)}")
                    conn.rollback()

        except Empty:
            if task_queue.empty() and response_queue.empty():
                print("📭 All queues empty. Finalizing...")
                break
            print("⏳ No responses received in last 30 seconds, continuing to wait...")
            continue
        except Exception as e:
            print(f"❌ Unexpected error in process_responses: {str(e)}")
            continue

    # Process any remaining updates
    if batch_updates:
        try:
            cursor.executemany("""
                UPDATE sep_embeddings 
                SET mistral_output = %s 
                WHERE id = %s;
            """, batch_updates)
            conn.commit()
            print(f"✅ Final batch uploaded ({len(batch_updates)} entries)")
        except Exception as e:
            print(f"❌ Error uploading final batch: {str(e)}")
            conn.rollback()

    print(f"\n🎉 Processing complete! Processed {processed_count} entries in {batch_count} batches")
    cursor.close()
    conn.close()


def run_api_workers():
    """Starts API workers with only 3 threads to reduce costs."""
    with ThreadPoolExecutor(MAX_WORKERS) as executor:
        for _ in range(MAX_WORKERS):  # Only 3 workers for cost efficiency
            executor.submit(api_worker)

# Execution
if __name__ == "__main__":
    # Start response processing thread
    response_thread = threading.Thread(target=process_responses)
    response_thread.start()

    # Run API workers
    run_api_workers()

    # Wait for response processing to complete
    response_thread.join()


🚀 Response processor started.
⏳ Waiting for responses... (Current batch size: 0)
📊 Stats - Processed: 0, Batches: 0
🔄 Processing entry 1924
📡 Calling API for entry 1924

🎯 Starting API call for entry 1924
⏳ Making API request for entry 1924...
🔄 Processing entry 11516
📡 Calling API for entry 11516

🎯 Starting API call for entry 11516
⏳ Making API request for entry 11516...
🔄 Processing entry 15945
📡 Calling API for entry 15945

🎯 Starting API call for entry 15945
⏳ Making API request for entry 15945...
🔄 Processing entry 33159
📡 Calling API for entry 33159

🎯 Starting API call for entry 33159
⏳ Making API request for entry 33159...
📡 Received response for entry 33159 (Status: 200)
🔍 Attempting to extract JSON for entry 33159

🔍 Raw API Content received:
{
    "category": "thinker",
    "metadata": {
        "name": "Sydney Shoemaker",
        "description": "Sydney Shoemaker is a prominent philosopher known for his work in the philosophy of mind, met...
ℹ️ Could not parse complete resp